In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as k
#from sklearnex import patch_sklearn
#patch_sklearn()
import sklearn
import os
import numba
import time
import json
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

In [2]:
#Loading Data

file=open("extra_words.json","r")
extra_words=json.load(file)
file.close()

file=open("vocab_final.json","r")
vocab_final=json.load(file)
file.close()

In [3]:
df=pd.read_json("data_ready.json")
data=pd.read_json("data_vector.json")

# Chapter 2

# Preparing the Corpus Data

We can easily do the below with pad sequences but that isn't fun. So, doing this.

In [4]:
import ipyparallel as ipp
import time
#global counting
#counting=0
c=ipp.Client()

dview=c[:]
#dview=dview.load_balanced_view()
#global vocab_final
##############################################################
# Function that converts each array into fixed length
##############################################################
#@dview.parallel(block=True)
#@keep_track
def word_tokens(list_passed,vocab_final=vocab_final):
    #print("entered function")
    #COUNTING+=1
    import numpy as np
    list_given=list_passed
    word_length=10 # Derived from Maximum length of the Words in the matrix of words
    tokens=[]
    final_array=[]
    #final_array=np.zeros(word_length,dtype=int)
    #print(final_array)
    for each in list_given:
        temp=[index for index,word in enumerate(vocab_final) if word==each]
        try:
            tokens.append(temp[0])
        except:
            pass
    #print(tokens)
    if len(tokens)>word_length:
        tokens=tokens[:word_length]
    else:
        zeros=[0]*(word_length-len(tokens))
        tokens=zeros+tokens
    final_array.append([t for t in tokens])
    #print(final_array)
    return np.array([item for item in final_array])
##############################################################
# End of Function
##############################################################


In [5]:
#df2["new_headline"]=df2["headline"].apply(word_tokens,vocab_final=vocab_final)

Now, we have fixed length sequences and we can subject these directly to the embeddings and training, with some of the numbers (having representing a word now), we can now feed it to our embeddings with our preloaded word embeddings given in the glove dataset along with the problem statement. Before that we will pad our sequences.

import json
file=open("file_word_index.json","r")
word_index_rev=json.load(file)
file.close()

## Another Approach (more simpler and faster one)

word_index=vocab_final

word_index_rev={value:int(key) for key,value in word_index.items()}

for each in irrelevant_words:
    try:
        _=word_index_rev.pop(each)
    except:
        pass

word_index={value:key for key,value in word_index_rev.items()}

file=open("file_word_index.json","w")
json.dump(word_index_rev,file)
file.close()

In [5]:
file=open("file_word_index.json","r")
word_index_rev=json.load(file)
file.close()

In [6]:
word_index_rev

{"'s": 9,
 'said': 16,
 'as': 19,
 "''": 27,
 '``': 28,
 'one': 48,
 'new': 50,
 'also': 52,
 'would': 54,
 'two': 55,
 'first': 58,
 'year': 62,
 '--': 65,
 'out': 66,
 'people': 69,
 "n't": 70,
 'percent': 72,
 'last': 76,
 'government': 78,
 'time': 79,
 'world': 85,
 'can': 86,
 'three': 87,
 'president': 90,
 'state': 92,
 'million': 93,
 'could': 94,
 'us': 95,
 'u.s.': 99,
 'united': 104,
 'may': 107,
 'since': 108,
 'many': 109,
 'city': 115,
 'made': 116,
 'like': 117,
 'national': 121,
 'day': 122,
 'country': 123,
 'second': 126,
 'company': 128,
 'group': 129,
 'china': 132,
 'four': 133,
 'being': 134,
 'down': 135,
 'war': 136,
 'back': 137,
 'south': 139,
 'american': 140,
 'minister': 141,
 'police': 142,
 'well': 143,
 'including': 144,
 'team': 145,
 'international': 146,
 'week': 147,
 'still': 149,
 'even': 151,
 'high': 152,
 'part': 153,
 'told': 154,
 'end': 156,
 'former': 157,
 'make': 159,
 'billion': 160,
 'work': 161,
 'home': 163,
 'school': 164,
 'party': 

In [7]:
df3=pd.DataFrame(df["headline"].tolist(),index=df.index)
#df3_a=df3[:10000]
#df3_b=df3[10000:20000]
#df3_c=df3[20000:]

In [8]:
l=df3[0:15].replace(to_replace=word_index_rev) #just a check how it is working out
l

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,157,20884,1612,7732,6415.0,1779.0,404085.0,2280.0,2342.0,26925.0,...,None,None,None,None,None,None,None,None,None,None
1,400829,5571,3161,18101,209.0,5030.0,439.0,2771.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,6075,1187,1655,630,9.0,1170.0,281.0,5275.0,873.0,32276.0,...,None,None,None,None,None,None,None,None,None,None
3,18205,303,702,4927,326.0,2488.0,402303.0,1159.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,27492,20564,3467,61891,1751.0,4158.0,10408.0,179.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
5,6561,85,9,787,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
6,12316,305,4525,401054,3806.0,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
7,3695,1545,4313,164,161.0,128.0,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
8,220,9517,22057,7501,7940.0,13539.0,398.0,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
9,185,9,766,12243,1038.0,10468.0,9.0,68923.0,464.0,NaN,...,None,None,None,None,None,None,None,None,None,None


In [9]:
df4=df3.replace(to_replace=word_index_rev)

In [10]:
df4

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,157.0,20884.0,1612.0,7732.0,6415.0,1779.0,404085.0,2280.0,2342.0,26925.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,400829.0,5571.0,3161.0,18101.0,209.0,5030.0,439.0,2771.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6075.0,1187.0,1655.0,630.0,9.0,1170.0,281.0,5275.0,873.0,32276.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18205.0,303.0,702.0,4927.0,326.0,2488.0,402303.0,1159.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27492.0,20564.0,3467.0,61891.0,1751.0,4158.0,10408.0,179.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26704,140.0,1378.0,4011.0,131386.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26705,453.0,9.0,254.0,6613.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26706,50716.0,765.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26707,406.0,1365.0,5627.0,5260.0,6240.0,1474.0,7203.0,2512.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df4=df4+1 # We do this to replace any zero values so that we can use zero as padding or "none" values
#df4.fillna(value=0,inplace=True)

In [12]:
#df4.to_json("final_vector_embeddings.json")
df4

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,158.0,20885.0,1613.0,7733.0,6416.0,1780.0,404086.0,2281.0,2343.0,26926.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,400830.0,5572.0,3162.0,18102.0,210.0,5031.0,440.0,2772.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6076.0,1188.0,1656.0,631.0,10.0,1171.0,282.0,5276.0,874.0,32277.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18206.0,304.0,703.0,4928.0,327.0,2489.0,402304.0,1160.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27493.0,20565.0,3468.0,61892.0,1752.0,4159.0,10409.0,180.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26704,141.0,1379.0,4012.0,131387.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26705,454.0,10.0,255.0,6614.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26706,50717.0,766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26707,407.0,1366.0,5628.0,5261.0,6241.0,1475.0,7204.0,2513.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Note**

We are also restricting ourselves to just 10 word sequences and chopping off the rest.

In [13]:
df4=df4.fillna(0).iloc[:,:10] #Just checking how it is going.
df4

,0,1,2,3,4,5,6,7,8,9
0,158.0,20885.0,1613.0,7733.0,6416.0,1780.0,404086.0,2281.0,2343.0,26926.0
1,400830.0,5572.0,3162.0,18102.0,210.0,5031.0,440.0,2772.0,0.0,0.0
2,6076.0,1188.0,1656.0,631.0,10.0,1171.0,282.0,5276.0,874.0,32277.0
3,18206.0,304.0,703.0,4928.0,327.0,2489.0,402304.0,1160.0,0.0,0.0
4,27493.0,20565.0,3468.0,61892.0,1752.0,4159.0,10409.0,180.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
26704,141.0,1379.0,4012.0,131387.0,0.0,0.0,0.0,0.0,0.0,0.0
26705,454.0,10.0,255.0,6614.0,0.0,0.0,0.0,0.0,0.0,0.0
26706,50717.0,766.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26707,407.0,1366.0,5628.0,5261.0,6241.0,1475.0,7204.0,2513.0,0.0,0.0


In [38]:
#df4["end"]=pd.Series([0]*df4.shape[0])
#df4 #To indicate end of Sequence acting as a pad.

,0,1,2,3,4,5,6,7,8,9,end
0,158.0,20885.0,1613.0,7733.0,6416.0,1780.0,404086.0,2281.0,2343.0,26926.0,1
1,400830.0,5572.0,3162.0,18102.0,210.0,5031.0,440.0,2772.0,0.0,0.0,1
2,6076.0,1188.0,1656.0,631.0,10.0,1171.0,282.0,5276.0,874.0,32277.0,1
3,18206.0,304.0,703.0,4928.0,327.0,2489.0,402304.0,1160.0,0.0,0.0,1
4,27493.0,20565.0,3468.0,61892.0,1752.0,4159.0,10409.0,180.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
26704,141.0,1379.0,4012.0,131387.0,0.0,0.0,0.0,0.0,0.0,0.0,1
26705,454.0,10.0,255.0,6614.0,0.0,0.0,0.0,0.0,0.0,0.0,1
26706,50717.0,766.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
26707,407.0,1366.0,5628.0,5261.0,6241.0,1475.0,7204.0,2513.0,0.0,0.0,1


Now our Entire corpus of the Dataframe is ready with its word coding given with coded words and appropriate numbers given. We now only need to supply this our embedding vector of the Neural Networks after padding it appropriately. But, before that let us also check our word embeddings once.
All these are 27 word sequences with fixed length of 10 and appropriate paddings done in later stages.

In [14]:
final_data=df4
final_data.loc[589]

0      165.0
1    17225.0
2     1596.0
3      109.0
4    25341.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
Name: 589, dtype: float64

# Preparing Word Embeddings

To save Bandwidth we have to drop some word embeddings which are absent in our corpus, but also include the news words in our corpus initializing them with zeros all together. So, we will create dataframes appropriately and mainpulate them. All the while we have to preserve our word and its corresponding vector index too.

In [109]:
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,-0.038194,-0.244870,0.728120,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,0.087459,...,0.016215,-0.017099,-0.389840,0.87424,-0.725690,-0.510580,-0.520280,-0.145900,0.82780,0.270620
",",-0.107670,0.110530,0.598120,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,-0.094189,...,0.349510,-0.722600,0.375490,0.44410,-0.990590,0.612140,-0.351110,-0.831550,0.45293,0.082577
.,-0.339790,0.209410,0.463480,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,-0.019169,...,-0.063351,-0.674120,-0.068895,0.53604,-0.877730,0.318020,-0.392420,-0.233940,0.47298,-0.028803
of,-0.152900,-0.242790,0.898370,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,0.425410,...,0.187120,-0.018488,-0.267570,0.72700,-0.593630,-0.348390,-0.560940,-0.591000,1.00390,0.206640
to,-0.189700,0.050024,0.190840,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,0.342410,...,-0.131340,0.058617,-0.318690,-0.61419,-0.623930,-0.415480,-0.038175,-0.398040,0.47647,-0.159830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chanty,-0.155770,-0.049188,-0.064377,0.223600,-0.201460,-0.038963,0.129710,-0.294510,0.00359,-0.098377,...,0.093324,0.094486,-0.023469,-0.48099,0.623320,0.024318,-0.275870,0.075044,-0.56380,0.145010
kronik,-0.094426,0.147250,-0.157390,0.071966,-0.298450,0.039432,0.021870,0.008041,-0.18682,-0.311010,...,-0.305450,-0.011082,0.118550,-0.11312,0.339510,-0.224490,0.257430,0.631430,-0.20090,-0.105420
rolonda,0.360880,-0.169190,-0.327040,0.098332,-0.429700,-0.188740,0.455560,0.285290,0.30340,-0.366830,...,-0.044082,0.140030,0.300070,-0.12731,-0.143040,-0.069396,0.281600,0.271390,-0.29188,0.161090
zsombor,-0.104610,-0.504700,-0.493310,0.135160,-0.363710,-0.447500,0.184290,-0.056510,0.40474,-0.725830,...,0.151530,-0.108420,0.340640,-0.40916,-0.081263,0.095315,0.150180,0.425270,-0.51250,-0.170540


In [15]:
extra_words_df=pd.DataFrame(np.zeros((len(extra_words),100)),index=extra_words)
extra_words_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
unicyclist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
network-tv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
porn-store,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'thank,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bejewel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jlaw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'justice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'life-changer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'gobble,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now drop the embeddings of the irrelevant_words form the original dataframe. This way we can go ahead.

In [16]:
dropped_irr_df=data.drop(irrelevant_words,axis=0)
dropped_irr_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
's,0.588540,-0.202500,0.734790,-0.68338,-0.196750,-0.180200,-0.391770,0.341720,-0.60561,0.638160,...,1.014900,0.776490,0.125980,0.54779,-1.03160,0.064599,-0.375230,-0.94475,0.618020,0.395910
said,-0.131280,-0.452000,0.043399,-0.99798,-0.210530,-0.958680,-0.246090,0.484130,0.18178,0.475000,...,1.062300,-0.628790,0.290980,0.13238,-0.70414,0.671280,-0.085462,-0.30526,-0.045495,0.565090
as,-0.327210,0.096446,0.342440,-0.44327,0.305350,-0.042016,-0.071235,-0.310360,-0.22557,-0.181000,...,0.049285,-0.253320,-0.307300,0.51295,-0.56802,-0.212390,-0.397410,-0.38165,0.439940,0.246830
'',0.164780,0.170710,0.621110,-1.21010,-0.840630,0.218930,0.481230,-0.150440,0.36701,-0.208570,...,0.065289,-0.567350,0.226210,0.47139,-0.48157,0.564380,0.340190,-0.44047,-0.105870,0.795090
``,0.092672,0.202410,0.693940,-0.50775,-0.097297,0.045522,-0.141560,0.307360,-0.35448,-0.206120,...,0.144990,-0.101270,0.252940,0.37802,-0.39809,-0.413800,-0.054480,-0.51348,-0.277500,1.163600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vlogger,0.186920,-0.005655,-0.026767,0.13866,0.311150,0.228040,0.102780,-0.670050,0.57540,0.190150,...,-0.424650,0.021242,-0.239300,-0.35385,0.50381,0.218140,-0.051292,-0.37233,-0.292530,-0.009761
whatsapp,-0.007094,-0.047403,-0.234780,-0.15492,0.086037,0.059698,0.006438,-0.282110,0.63746,-0.104690,...,-0.079207,0.145880,0.201920,-0.34398,0.52091,-0.133880,0.016467,0.35578,-0.431810,0.121860
gas-electric,0.086142,0.222960,-0.253530,0.52736,0.306490,-0.088289,0.767800,-0.179450,0.36555,0.148460,...,0.164750,0.102700,-0.092577,-0.64699,0.23623,0.069959,-0.316960,0.35109,0.048252,-0.067979
shot-for-shot,-0.018148,-0.000653,-0.103940,0.58740,0.411750,0.585470,0.154190,-0.046174,0.32092,0.605890,...,0.475230,0.516130,-0.502620,-0.43755,0.82111,-0.584460,0.395650,0.33204,-0.317200,-0.186950


In [17]:
dropped_irr_df.to_json("relevant_words.json")

This will be useful later on if we want to go for cross referencing. So, lets keep it aside and switch to our original data frame.

In [18]:
final_word_embeddings=pd.concat([data,extra_words_df])

In [19]:
final_word_embeddings

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,0.087459,...,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,-0.094189,...,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,-0.019169,...,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,0.425410,...,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,0.342410,...,-0.131340,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jlaw,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
'justice,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
'life-changer,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
'gobble,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000


A Little more processing and here we are done with this particular code and we will move towards the next chapter.

In [20]:
final_word_embeddings.index.name="words"

In [21]:
final_word_embeddings=final_word_embeddings.reset_index()

In [22]:
final_word_embeddings

,words,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,...,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
1,",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,...,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
2,.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,...,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
3,of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,...,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
4,to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,...,-0.131340,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404200,jlaw,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404201,'justice,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404202,'life-changer,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404203,'gobble,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000


Let us now finally finish this chapter by saving both our corpus table and our final word embeddings and move to chapter 3 i.e. designing our LSTM model that is going to train on this.

**Please Note**
Due to computational restraint on the system, I am continuing with this trend of only 10000 entries of the given dataset, i.e. only half of the data set, which can easily be removed at any time.
Also note that padding has already be done in this chapter.

In [23]:
final_word_embeddings.to_json("final_word_embeddings.json")
#
#
#
pd.DataFrame(final_data).to_json("final_corpus_data.json")


# End of Chapter 2